# Deep Learning
## 神经网络 Neural Network Introduction
### 反向传播

- sigmoid 函数导数的最大值是 0.25，因此输出层的误差被减少了至少 75%，隐藏层的误差被减少了至少 93.75%！
- 如果你的神经网络有很多层，使用 sigmoid 激活函数会很快把靠近输入层的权重步长降为很小的值，该问题称作梯度消失。

In [7]:
import numpy as np
a = np.array([1,2,3])
print a.reshape(-1,1)
print np.dot(a, a.reshape(-1,1))
print np.dot(a, a)
print np.dot(a.reshape(-1,1), a.reshape(1,-1))

[[1]
 [2]
 [3]]
[14]
14
[[1 2 3]
 [2 4 6]
 [3 6 9]]


In [12]:
b = np.array([1,2,3])
print b * (b-1)
print np.multiply(b, (b-1))

[0 2 6]
[0 2 6]


## TensorFlow Introduction
### TensorFlow 环境安装

#### OS X 或 Linux

运行下列命令来配置开发环境
```bash
conda create -n tensorflow python=3.5
source activate tensorflow
conda install pandas matplotlib jupyter notebook scipy scikit-learn
conda install -c conda-forge tensorflow
```

- 从正态分布中取随机数来初始化权重是个好习惯。随机化权重可以避免模型每次训练时候卡在同一个地方。
- 类似地，从正态分布中选择权重可以避免任意一个权重与其他权重相比有压倒性的特性。你可以用 `tf.truncated_normal() `函数从一个正态分布中生成随机数。
### Relu 和 Softmax 函数
- S 型函数的导数最大值为 0.25（如上所示）。这意味着，当你用 S 型函数单元进行反向传播时，网络上每层出现的错误至少减少 75%，如果有很多层，权重更新将很小，这些权重需要很长的训练时间。因此，S 型函数不适合作为隐藏单元上的激活函数。
- ReLU 激活函数是你可以使用的最简单非线性激活函数。当输入是正数时，导数是 1，所以没有 S 型函数的反向传播错误导致的消失效果。研究表明，对于大型神经网络来说，ReLU 的训练速度要快很多。

摘自 [Andrej Karpathy 的 CS231n 课程](http://cs231n.github.io/neural-networks-1/#nn):
> 遗憾的是，ReLU 单元在训练期间可能会很脆弱并且会变得“无效”。例如，流经 ReLU 神经元的大型梯度可能会导致权重按以下方式更新：神经元将再也不会在任何数据点上激活。如果发生这种情况，那么流经该单元的梯度将自此始终为零。也就是说，ReLU 单元会在训练期间变得无效并且不可逆转，因为它们可能会不再位于数据流形上。例如，学习速度（learning rate）设置的太高，你的网络可能有高达 40% 的神经元处于“无效”状态（即神经元在整个训练数据集上从未激活）。如果能正确地设置学习速度，那么该问题就不太容易出现。

#### Softmax

- 和 sigmoid 一样，softmax 函数将每个单元的输出压缩到 0 和 1 之间。但 softmax 函数在拆分输出时，会使输出之和等于 1。**softmax 函数的输出等于分类概率分布，显示了任何类别为真的概率。**我们将会使用 softmax 函数。和 sigmoid 一样，softmax 函数将每个单元的输出压缩到 0 和 1 之间。但 softmax 函数在拆分输出时，会使输出之和等于 1。softmax 函数的输出等于分类概率分布，显示了任何类别为真的概率。
- softmax 函数与普通 sigmoid 之间的真正差别是 softmax 会标准化输出，使输出之和等于 1。对于这两种函数，你都可以输入向量，并获得输出为相同大小的向量，但是所有值都压缩在 0 和 1 之间。sigmoid 可用于只有一个输出单元的二元分类。但是如果进行多项分类的话，则需要使用多个输出单元（每个类别一个单元），并对输出进行 softmax 激活。
- softmax 可以用于任何数量的分类。接下来你将看到，它可以用于预测两种类别的情感（积极和消极）。还可以用于成百上千的物体分类，例如物体识别问题中，需要识别数以百计不同种类的物体。

### 独热编码 One Hot Encoding

用 `Scikit-Learn` 实现 One-Hot Encoding
`scikit-learn` 的 `LabelBinarizer` 函数可以很方便地把你的目标（labels）转化成独热编码向量。
```python
import numpy as np
from sklearn import preprocessing

# Example labels 示例 labels
labels = np.array([1,5,3,2,1,4,2,1,3])

# Create the encoder 创建编码器
lb = preprocessing.LabelBinarizer()

# Here the encoder finds the classes and assigns one-hot vectors 
# 编码器找到类别并分配 one-hot 向量
lb.fit(labels)

# And finally, transform the labels into one-hot encoded vectors
# 最后把目标（lables）转换成独热编码的（one-hot encoded）向量
lb.transform(labels)
>>> array([[1, 0, 0, 0, 0],
           [0, 0, 0, 0, 1],
           [0, 0, 1, 0, 0],
           [0, 1, 0, 0, 0],
           [1, 0, 0, 0, 0],
           [0, 0, 0, 1, 0],
           [0, 1, 0, 0, 0],
           [1, 0, 0, 0, 0],
           [0, 0, 1, 0, 0]])
```

### TensorFlow 中的交叉熵（Cross Entropy）

![cross entropy](img/cross_entropy.jpg)

### 计算稳定性
数值计算时，当心数值运算上溢或下溢。如当一个很大的数（比如1e9）加上一个很小的数（比如1e-6）：
```python
a = 1000000000
for i in range(1000000):
    a = a + 1e-6
print(a - 1000000000)

>>> 0.953674316406
```

正因为如此，我们在做权重系数初始化的时候，应尽量使得初始化变量保持均值为0，且具有相同的方差。这样做的目的不光能避免数值溢出，还能有助于优化器更容易找到最小值点。

![归一化输入和权重](img/normalized_inputs_and_initial_weights.jpg)

为此，在做图像识别时，针对图像输入，我们一般会做出如下处理：

![图像输入归一化](img/image_inputs_normalized.jpg)

### 随机梯度下降法 Stochastic Gradient Descent

用于解决梯度下降法难于规模化的问题。

### 动量与学习率衰减 Momentum And Learning Rate Decay

动量法：考虑上一步的梯度方向，追踪梯度的实时平均（running average）

### 参数超空间

- High Learning Rate DONT mean learning fast or learning more
- 当学习效果不好的时候，首先要做的事情是降低学习率
- ADAGRAD是SGD的改良版

In [3]:
784*10*4

31360

In [6]:
print 50000/128
print 50000%128

390
80


In [13]:
lst = range(11)
for i in range(0,11,3):
    print(i)
    print lst[i:i+3]

0
[0, 1, 2]
3
[3, 4, 5]
6
[6, 7, 8]
9
[9, 10]
